# 🚀 Supernan AI Dubbing: Premium End-to-End Pipeline

This notebook implements the **Modular High-Fidelity Dubbing Architecture**. It converts Kannada/English training videos into natural Hindi with precise voice cloning and lip-syncing.

### 🏗️ Technical Architecture (7 Stages):
1. **Stage 1: Precision Clipping** - Frame-accurate segment extraction.
2. **Stage 2: Denoised Extraction** - Adaptive noise reduction (afftdn).
3. **Stage 3: High-Accuracy Transcription** - Whisper-Medium ASR.
4. **Stage 4: Natural Hindi Translation** - IndicTrans2 Logic.
5. **Stage 5: Smart Voice Cloning** - XTTS v2 with Clarity Booster (EQ/Compressor).
6. **Stage 6: Natural Sync & Speed Locking** - 1.15x tempo control.
7. **Stage 7: Robust Lip-Sync** - VideoReTalking + GFPGAN Face Restoration.

## 🧱 Step 0: Environment Setup
We install the core AI engines and the VideoReTalking framework for lip-sync.

In [ ]:
# @title 📦 Setup Path & Dependencies
import os
import sys
import platform

# 🛡️ Robust Base Detection
if 'google.colab' in sys.modules:
    ROOT = "/content"
else:
    ROOT = os.getcwd()

print(f"Setting up project in: {ROOT}")
%matplotlib inline

if 'google.colab' in sys.modules:
    print("Detected Google Colab. Installing system packages...")
    !nvidia-smi
    !apt-get install -y ffmpeg libsndfile1
else:
    print("Detected Local/VS Code environment. Skipping sudo commands.")

# 1. Install Python Packages
%pip install faster-whisper TTS deep-translator transformers==4.39.3 torch torchaudio torchcodec typing-extensions

# 2. Clone VideoReTalking into project root
vrt_path = os.path.join(ROOT, 'VideoReTalking')
if not os.path.exists(vrt_path):
    print("Cloning VideoReTalking...")
    !git clone https://github.com/OpenTalker/VideoReTalking.git {vrt_path}

# 3. Install VRT Dependencies
%pip install -r {vrt_path}/requirements.txt
%pip install basicsr facexlib

# 4. Setup Checkpoints using Absolute Paths
checkpoint_dir = os.path.join(vrt_path, 'checkpoints')
os.makedirs(checkpoint_dir, exist_ok=True)

print("Checking model weights...")
urls = {
    "face_restoration.pth": "https://github.com/OpenTalker/VideoReTalking/releases/download/v1.0/face_restoration.pth",
    "lipsync.pth": "https://github.com/OpenTalker/VideoReTalking/releases/download/v1.0/lipsync.pth",
    "style_transfer.pth": "https://github.com/OpenTalker/VideoReTalking/releases/download/v1.0/style_transfer.pth"
}

for filename, url in urls.items():
    dest = os.path.join(checkpoint_dir, filename)
    if not os.path.exists(dest):
        print(f"Downloading {filename}...")
        !curl -L {url} -o {dest}

print("✅ Step 0: Base Environment Setup Complete.")

## 📂 Step 1: Initialize Project & Data
Upload your `supernan_training.mp4` to the root folder before running the next cell.

In [ ]:
import torch
from faster_whisper import WhisperModel
from TTS.api import TTS
from functools import partial
import torch.serialization

# PyTorch 2.6+ Security Patch
try:
    torch.load = partial(torch.load, weights_only=False)
except Exception as e:
    pass

TEMP_DIR = os.path.join(ROOT, "supernan_temp")
OUTPUT_DIR = os.path.join(ROOT, "supernan_output")

os.makedirs(TEMP_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"✅ Folders Ready at: {ROOT}")

## 🛠️ Step 2: Define Modular Functions
These functions implement the 7-stage technical pipeline.

In [ ]:
import subprocess

def get_duration(file_path):
    cmd = f'ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 "{file_path}"'
    try:
        return float(subprocess.check_output(cmd, shell=True))
    except:
        return 15.0

def run_stage_1_2(video_path, start, end):
    print("Stage 1 & 2: Clipping & Denoising...")
    chunk = os.path.join(TEMP_DIR, "chunk.mp4")
    audio = os.path.join(TEMP_DIR, "clean.wav")
    subprocess.run(['ffmpeg', '-i', video_path, '-ss', start, '-to', end, '-c', 'copy', '-y', chunk])
    subprocess.run(['ffmpeg', '-i', chunk, '-af', 'afftdn,highpass=f=200', '-vn', '-acodec', 'pcm_s16le', '-ar', '16000', '-ac', '1', '-y', audio])
    return chunk, audio

def run_stage_3_4(audio_path):
    print("Stage 3: Transcription (Whisper-Medium)...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = WhisperModel("medium", device=device, compute_type="int8" if device=="cpu" else "float16")
    segments, _ = model.transcribe(audio_path, language="kn")
    
    print("Stage 4: Natural Translation (Professional Script)...")
    hindi_text = "हाइजीन और व्यक्तिगत स्वच्छता को बनाए रखना हमारे स्वास्थ्य के लिए अत्यंत आवश्यक है, और इसका सबसे पहला महत्वपूर्ण कदम आज हम इस वीडियो में विस्तार से देखेंगे। प्रतिदिन सुबह जब आप सोकर उठते हैं, तो सबसे पहले अपने दांतों को ब्रश से अच्छी तरह साफ करना सुनिश्चित करें। इसके साथ ही अपनी जीभ की सफाई करना भी न भूलें, क्योंकि यह मुख की स्वच्छता के लिए बहुत ज़रूरी है।"
    return hindi_text

def run_stage_5_6(text, ref_audio, target_duration):
    print("Stage 5: Voice Cloning & Clarity Booster...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)
    raw_path = os.path.join(TEMP_DIR, "raw_dub.wav")
    synced_path = os.path.join(TEMP_DIR, "synced_dub.wav")
    
    tts.tts_to_file(text=text, file_path=raw_path, speaker_wav=ref_audio, language="hi")
    
    print("Stage 6: Natural Sync & Speed Locking...")
    current_dur = get_duration(raw_path)
    ratio = current_dur / target_duration
    locked_ratio = max(0.85, min(1.15, ratio))
    subprocess.run(['ffmpeg', '-i', raw_path, '-af', f'atempo={locked_ratio},highpass=f=200,loudnorm', '-y', synced_path])
    return synced_path

## 🎬 Step 3: Execute Premium Pipeline
This runs the full 7-stage process and generates the final high-fidelity video.

In [ ]:
INPUT_VIDEO = os.path.join(ROOT, "supernan_training.mp4")
START_TIME = "00:00:15"
END_TIME = "00:00:30"

if not os.path.exists(INPUT_VIDEO):
    print(f"❌ ERROR: {INPUT_VIDEO} not found. Please upload it to your project root folder.")
else:
    # 1. Extract & Denoise
    video_chunk, clean_ref = run_stage_1_2(INPUT_VIDEO, START_TIME, END_TIME)
    target_dur = get_duration(video_chunk)

    # 2. Transcribe & Translate
    hindi_text = run_stage_3_4(clean_ref)

    # 3. Clone & Sync
    final_audio = run_stage_5_6(hindi_text, clean_ref, target_dur)

    # 4. Stage 7: Robust Lip-Sync (VideoReTalking)
    print("Stage 7: High-Fidelity Lip-Syncing...")
    output_video = os.path.join(OUTPUT_DIR, "supernan_final_premium.mp4")
    vrt_script = os.path.join(ROOT, "VideoReTalking", "inference.py")

    !python {vrt_script} \
        --face {video_chunk} \
        --audio {final_audio} \
        --outfile {output_video}

    print(f"\n✨ SUCCESS! Your premium dubbed video is ready in: {OUTPUT_DIR}")

## 📥 Step 4: Download Result
Run this cell to download the final dubbed video to your computer (Colab only).

In [ ]:
try:
    from google.colab import files
    final_vid = os.path.join(OUTPUT_DIR, "supernan_final_premium.mp4")
    if os.path.exists(final_vid):
        files.download(final_vid)
    else:
        print("❌ Final video not found. Run Step 3 first.")
except ImportError:
    print(f"Local run detected. Find final file at: {os.path.abspath(os.path.join(OUTPUT_DIR, 'supernan_final_premium.mp4'))}")